# 文本分类实例

## 1. 导入相关包

In [35]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

## 2. 加载数据

In [1]:
import pandas as pd

In [12]:
data = pd.read_csv("./ChnSentiCorp_htl_all.csv")
data.head()

,label,review
0,1,"距离川沙公路较近,但是公交指示不对,如果是""蔡陆线""的话,会非常麻烦.建议用别的路线.房间较..."
1,1,商务大床房，房间很大，床有2M宽，整体感觉经济实惠不错!
2,1,早餐太差，无论去多少人，那边也不加食品的。酒店应该重视一下这个问题了。房间本身很好。
3,1,宾馆在小街道上，不大好找，但还好北京热心同胞很多~宾馆设施跟介绍的差不多，房间很小，确实挺小...
4,1,"CBD中心,周围没什么店铺,说5星有点勉强.不知道为什么卫生间没有电吹风"


去除Null

In [13]:
data.shape

(7766, 2)

In [15]:
data = data.dropna()
data.shape

(7765, 2)

## 3. 创建dataset

In [20]:
%config Completer.use_jedi = False

In [21]:
from torch.utils.data import Dataset

In [26]:
class MyDataset(Dataset):

    def __init__(self) -> None:
        super().__init__()
        self.data = pd.read_csv("./ChnSentiCorp_htl_all.csv")
        self.data = self.data.dropna()

    def __getitem__(self, index):
        return self.data.iloc[index]["review"], self.data.iloc[index]["label"]
    
    def __len__(self):
        return len(self.data)

In [27]:
dataset = MyDataset()
for i in range(5):
    print(dataset[i])

('距离川沙公路较近,但是公交指示不对,如果是"蔡陆线"的话,会非常麻烦.建议用别的路线.房间较为简单.', 1)
('商务大床房，房间很大，床有2M宽，整体感觉经济实惠不错!', 1)
('早餐太差，无论去多少人，那边也不加食品的。酒店应该重视一下这个问题了。房间本身很好。', 1)
('宾馆在小街道上，不大好找，但还好北京热心同胞很多~宾馆设施跟介绍的差不多，房间很小，确实挺小，但加上低价位因素，还是无超所值的；环境不错，就在小胡同内，安静整洁，暖气好足-_-||。。。呵还有一大优势就是从宾馆出发，步行不到十分钟就可以到梅兰芳故居等等，京味小胡同，北海距离好近呢。总之，不错。推荐给节约消费的自助游朋友~比较划算，附近特色小吃很多~', 1)
('CBD中心,周围没什么店铺,说5星有点勉强.不知道为什么卫生间没有电吹风', 1)


## 4. 划分数据集

In [32]:
from torch.utils.data import random_split

total_len = dataset.__len__()
trainset_len = (int) (total_len * 0.9)
validset_len = total_len - trainset_len
trainset, validset = random_split(dataset, lengths=[trainset_len, validset_len])
len(trainset), len(validset)

(6988, 777)

In [33]:
for i in range(10):
    print(trainset[i])

('服务很好，房间也很明亮，性价比还算不错。', 1)
('酒店位置、房间都很不错，价格也比较合适，推荐大家去住', 1)
('从永州回到长沙，由于是学生返回的高峰期，酒店需要信用卡担保，但是由于我丢了钱包还没有补办好，因此只有到酒店前台进行确认有无房间。到了前台服务人员告知有房间，通过携程预定后，但是前台小姐的脸色又非常不好看了，直接告诉我，办理办理一张他们的酒店卡，比携程的方便（我靠，我又不是傻子），这个时候有一个客人来了，小姐依然用同样的话告诉他，这个客户同意了，我就在旁边看，结果和我预定是相同的房间，的确便宜20元，但是根本没有给这个客户一张会员卡，然后这个服务元很傲气地斜眼看了我，但是那个郁闷哦！又在大厅等待了20分钟，多次催促小姐去拿传真，根本不理睬你，最后我发飙要向长沙旅游局投诉，这才给我办理手续（该前台服务员姓刘）。由于上次入住没有了经验，我这次要求楼层要高，而且不能够窗户对准51大道，避免吵闹，给我了1820号房间，进去感觉房间比上次的8901房间好一些，但是窗帘关闭不严的毛病依然存在。第二天从客户那里回来，已经是晚上8点多了，打开房间一看，结果发现依然没有打扫，呵呵，我感觉这家酒店对携程的客户估计当作二等房客来对待的，希望携程和这个酒店好好沟通一下。', 0)
('酒店坐落的位置很好,闲暇在湖边散步很是惬意.酒店的前厅部人员很热情,去了酒店几次，感觉都不错。', 1)
('酒店服务还不错,在宜宾已经很好了.但是携程的价格一点都没有,豪标B在前台打折价265,携程263,希望以后携程要大大改进价格偏偏高的问题,才能有吸引了!!!', 1)
('酒店设施老,服务态度差,这么热的天不开空调,动不动就说你们是优惠房价,不享受空调是应该的,其实整栋楼是因为入住率低才不舍得开空调的,当然泳池也不会开放的.根本没有达到4星级的服务标准,更别说服务意识了.提醒大家千万别看以前的点评,估计是他们自己人写的.因为360的饭费不包括早餐,前台竟然说如果我们付360的现金就可以早餐优惠10元,试想这是正规的酒店么?房间发票开好再想起没有查房,谁还高兴等他们查房,自然一走了之了,早知道,怎么也带点东西走呢.宾馆反馈2008年7月28日：尊敬的客人，首先我们感谢您选择入住本酒店并提出宝贵意见。对于你在住店期间的不愉快感受，我们表示诚挚的歉意。酒店对您的感受非常重视，已召开了

## 5. 创建DataLoader

In [42]:
import torch

tokenizer = AutoTokenizer.from_pretrained("./rbt3")

def collate_func(batch):
    texts, labels = [], []
    for item in batch:
        texts.append(item[0])
        labels.append(item[1])
    inputs = tokenizer(texts, max_length=128, padding="max_length", truncation=True, return_tensors="pt")
    inputs["labels"] = torch.tensor(labels)
    return inputs

In [43]:
from torch.utils.data import DataLoader
# trainloader = DataLoader(trainset, batch_size=32, shuffle = True)
# validloader = DataLoader(validset, batch_size=64, shuffle = False)
# 此时数据还是文字，而label是tensor
# 需要collate function处理数据为tensor
trainloader = DataLoader(trainset, batch_size=32, shuffle=True, collate_fn=collate_func)
validloader = DataLoader(validset, batch_size=64, shuffle=False, collate_fn=collate_func)

In [44]:
# 查看trainloader
# next(enumerate(trainloader)) # 返回一个tuple[batch number, [data, label]]
next(enumerate(trainloader))[1]

{'input_ids': tensor([[ 101, 2791, 7313,  ..., 4510, 6228,  102],
        [ 101, 3297, 4289,  ..., 1220,  711,  102],
        [ 101, 1963, 3362,  ..., 1391, 3193,  102],
        ...,
        [ 101, 1765, 4157,  ...,    0,    0,    0],
        [ 101, 2769, 1343,  ...,    0,    0,    0],
        [ 101, 2769, 1762,  ..., 1862,  679,  102]]), 'token_type_ids': tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 1, 1, 1]]), 'labels': tensor([1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 0, 1, 0, 0])}

## 6. 创建模型及优化器

In [46]:
from torch.optim import Adam

model = AutoModelForSequenceClassification.from_pretrained("./rbt3")

if torch.cuda.is_available():
    model = model.cuda()

Some weights of the model checkpoint at ./rbt3 were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at ./rbt3 and

In [47]:
optimizer = Adam(model.parameters(), lr=2e-5)

## 7. 训练与验证

In [48]:
def evaluate():
    model.eval()
    acc_num = 0
    with torch.inference_mode():
        for batch in validloader:
            if torch.cuda.is_available():
                batch = {k: v.cuda() for k, v in batch.items()}
            output = model(**batch)
            pred = torch.argmax(output.logits, dim=-1)
            acc_num += (pred.long() == batch["labels"].long()).float().sum()
    return acc_num / len(validset)

def train(epoch=3, log_step=100):
    global_step = 0
    for ep in range(epoch):
        model.train()
        for batch in trainloader:
            if torch.cuda.is_available():
                batch = {k: v.cuda() for k, v in batch.items()}
            optimizer.zero_grad()
            output = model(**batch)
            output.loss.backward()
            optimizer.step()
            if global_step % log_step == 0:
                print(f"ep: {ep}, global_step: {global_step}, loss: {output.loss.item()}")
            global_step += 1
        acc = evaluate()
        print(f"ep: {ep}, acc: {acc}")

## 8.训练模型

In [49]:
train()

ep: 0, global_step: 0, loss: 0.9110406041145325
ep: 0, global_step: 100, loss: 0.20251959562301636
ep: 0, global_step: 200, loss: 0.3933698236942291
ep: 0, acc: 0.8931788802146912
ep: 1, global_step: 300, loss: 0.5275416374206543
ep: 1, global_step: 400, loss: 0.26682645082473755
ep: 1, acc: 0.9086229205131531
ep: 2, global_step: 500, loss: 0.16997507214546204
ep: 2, global_step: 600, loss: 0.28405457735061646
ep: 2, acc: 0.8944659233093262


# 9. 模型预测

In [55]:
sen = "我觉得这家酒店不错，饭很好吃！"
id2_label = {0: "差评！", 1: "好评！"}
model.eval()
with torch.inference_mode():
    inputs = tokenizer(sen, return_tensors="pt")
    inputs = {k: v.cuda() for k, v in inputs.items()}
    logits = model(**inputs).logits
    pred = torch.argmax(logits, dim = -1)
    print(f"输入：{sen}\n模型预测结果:{id2_label.get(pred.item())}")

输入：我觉得这家酒店不错，饭很好吃！
模型预测结果:好评！


In [56]:
from transformers import pipeline

model.config.id2label = id2_label
pipe = pipeline("text-classification", model=model, tokenizer=tokenizer, device=0)

In [57]:
pipe(sen)

[{'label': '好评！', 'score': 0.9969170093536377}]